In [ ]:
# Exercise 2.2.
# Wine Quality Data Set: "data/wines.csv"
# source: https://archive.ics.uci.edu/ml/datasets/wine+quality
# The file contains data on samples of white and red Portuguese wine
# Vinho Verde.
# Various physico-chemical characteristics of individual samples
# are available as well as wine quality scores on a point scale (0-10)
# made by specialists.

# Re-run your best models for all algorithms for 5-fold CV.
# Check the stability of results for repeated K-fold
# Check in repeated k-fold CV if adding stratification changes your results (stability).
# Compare the effect of stratification with titanic problem.
# Check if you didnt overfit in your models. Check if you can imrpove you validation score.

In [1]:
import pandas as pd
import numpy as np
import pickle
import statsmodels.api as sm
import matplotlib.pyplot as plt
import gc
from sklearn.metrics import roc_auc_score

In [3]:
df = pd.read_csv("wines.csv")

In [4]:
df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,type,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,red,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,red,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,red,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,red,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,red,5


In [5]:
df['quality'].unique()

array([5, 6, 7, 4, 8, 3, 9])

In [6]:
df.isnull().sum()

,0
fixed_acidity,0
volatile_acidity,0
citric_acid,0
residual_sugar,0
chlorides,0
free_sulfur_dioxide,0
total_sulfur_dioxide,0
density,0
pH,0
sulphates,0


In [8]:
mod = sm.GLM.from_formula(formula = 'quality ~ fixed_acidity + volatile_acidity + citric_acid + residual_sugar + chlorides + free_sulfur_dioxide + total_sulfur_dioxide + density + pH + sulphates + alcohol + C(type)',
                          data = df,
                          family = sm.families.Binomial()) # Note: Binomial family expects a binary dependent variable. 'quality' should be binarized if this family is intended.
res = mod.fit()
res.summary()

/usr/local/lib/python3.12/dist-packages/statsmodels/genmod/families/family.py:1056: RuntimeWarning: invalid value encountered in log
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu + 1e-20)) +
/usr/local/lib/python3.12/dist-packages/statsmodels/genmod/families/family.py:1057: RuntimeWarning: invalid value encountered in log
  n * np.log(1 - mu + 1e-20)) * var_weights


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                quality   No. Observations:                 6497
Model:                            GLM   Df Residuals:                     6484
Model Family:                Binomial   Df Model:                           12
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                   -inf
Date:                Wed, 14 Jan 2026   Deviance:                   2.3907e+06
Time:                        03:28:00   Pearson chi2:                 7.02e+20
No. Iterations:                     2   Pseudo R-squ. (CS):                nan
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept              7.02e+17   1.29e+09   5.43e+08      0.000    7.02e+17    7.02e+17
C(type)[T.white]     -2.441e+15    5.2e+06   -4.7e+08      0.000   -2.44e+15   -2.44e+15
fixed_acidity         5.747e+14   1.44e+06   3.98e+08      0.000    5.75e+14    5.75e+14
volatile_acidity     -1.008e+16   7.45e+06  -1.35e+09      0.000   -1.01e+16   -1.01e+16
citric_acid           -4.23e+14    7.3e+06   -5.8e+07      0.000   -4.23e+14   -4.23e+14
residual_sugar        4.218e+14   5.43e+05   7.76e+08      0.000    4.22e+14    4.22e+14
chlorides            -5.116e+15   3.06e+07  -1.67e+08      0.000   -5.12e+15   -5.12e+15
free_sulfur_dioxide   3.335e+13   7.01e+04   4.75e+08      0.000    3.34e+13    3.34e+13
total_sulfur_dioxide -9.476e+12   2.96e+04   -3.2e+08      0.000   -9.48e+12   -9.48e+12
density               -7.02e+17   1.31e+09  -5.35e+08      0.000   -7.02e+17   -7.02e+17
pH                    3.369e+15   8.29e+06   4.06e+08      0.000    3.37e+15    3.37e+15
sulphates             4.876e+15   6.98e+06   6.99e+08      0.000    4.88e+15    4.88e+15
alcohol               1.504e+15   1.65e+06   9.09e+08      0.000     1.5e+15     1.5e+15
========================================================================================
"""

In [11]:
from sklearn.model_selection import train_test_split
import random

# Create a binary target variable from 'quality'
# For example, let's define 'good quality' as >= 7
df['is_good_quality'] = (df['quality'] >= 7).astype(int)

X_train, X_test, y_train, y_test = train_test_split(df,
                                                    df.is_good_quality, # Use the new binary target
                                                    test_size = 0.3,
                                                    random_state = random.randint(0, 1000))
print(X_train.shape, X_test.shape)

mod = sm.GLM.from_formula(formula = 'is_good_quality ~ fixed_acidity + volatile_acidity + citric_acid + residual_sugar + chlorides + free_sulfur_dioxide + total_sulfur_dioxide + density + pH + sulphates + alcohol + C(type)',
                          data = X_train, # Train on X_train, not the full df
                          family = sm.families.Binomial())
res = mod.fit()
res.summary()

preds = res.predict(X_test)

roc_auc_score(y_test, preds)

(4547, 14) (1950, 14)


np.float64(0.7989176554019977)

In [18]:
scores = []

for k in range(10) :

    X_train, X_test, y_train, y_test = train_test_split(df,
                                                        df.is_good_quality,
                                                        stratify = df.is_good_quality,
                                                        test_size = 0.3,
                                                        random_state = random.randint(0, 10000))
    # print(X_train.shape, X_test.shape)
    mod = sm.GLM.from_formula(formula = 'is_good_quality ~ fixed_acidity + volatile_acidity + citric_acid + residual_sugar + chlorides + free_sulfur_dioxide + total_sulfur_dioxide + density + pH + sulphates + alcohol + C(type)',
                          data = X_train, # Train on X_train, not the full df
                          family = sm.families.Binomial())
    res = mod.fit()
    res.summary()
    preds = res.predict(X_test)
    scores.append(roc_auc_score(y_test, preds))

In [19]:
print(scores)

[np.float64(0.7972427398648029), np.float64(0.7954015672461223), np.float64(0.8133967385418246), np.float64(0.8041908754484214), np.float64(0.80177985573871), np.float64(0.8359690149809802), np.float64(0.815201254330088), np.float64(0.808378085213801), np.float64(0.808829630715758), np.float64(0.7987606658879869)]


In [21]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 10, shuffle = True, random_state = random.randint(0, 10000))

for train, test in kf.split(df.index.values) :

    mod = sm.GLM.from_formula(formula = 'is_good_quality ~ fixed_acidity + volatile_acidity + citric_acid + residual_sugar + chlorides + free_sulfur_dioxide + total_sulfur_dioxide + density + pH + sulphates + alcohol + C(type)',
                          data = df.iloc[train], # Train on X_train, not the full df
                          family = sm.families.Binomial())
    res = mod.fit()
    predsTrain = res.predict(df.iloc[train])
    preds = res.predict(df.iloc[test])
    print("Train AUC:", round(roc_auc_score(df.iloc[train].is_good_quality, predsTrain), 4), "Valid AUC:",
          round(roc_auc_score(df.iloc[test].is_good_quality, preds), 4))

Train AUC: 0.8134 Valid AUC: 0.7999
Train AUC: 0.8132 Valid AUC: 0.8038
Train AUC: 0.813 Valid AUC: 0.8097
Train AUC: 0.808 Valid AUC: 0.8544
Train AUC: 0.8162 Valid AUC: 0.7803
Train AUC: 0.8153 Valid AUC: 0.7833
Train AUC: 0.8147 Valid AUC: 0.7905
Train AUC: 0.813 Valid AUC: 0.807
Train AUC: 0.8095 Valid AUC: 0.8394
Train AUC: 0.8102 Valid AUC: 0.8359
